In [2]:
# Dependencies
import requests
import json
from citipy import citipy
import random
import openweathermapy.core as owm
# from config import api_key

1. Get random coordinate pairs, max lat is +-90, max long is +-180
2. Append into list (coordinates)

In [38]:
# Some random coordinates
coordinates = []

# Create random coordinates and append to coordinates list
for x in range(10):
    coordinates.append((random.randint(-90, 91), random.randint(-180, 181)))

print(coordinates)

[(-47, -86), (-87, -124), (-43, -96), (79, -110), (-73, 7), (-85, 103), (-33, 156), (-37, -83), (61, -106), (26, -118)]


In [39]:
cities = []
for coordinate_pair in coordinates:
    lat, lon = coordinate_pair
    city_obj = citipy.nearest_city(lat, lon)
    cities.append(city_obj.city_name)

print(cities)

['castro', 'punta arenas', 'castro', 'yellowknife', 'hermanus', 'albany', 'port macquarie', 'lebu', 'la ronge', 'guerrero negro']


In [35]:
for city in cities:
    country_code = city.country_code
    name = city.city_name
    print(f"The country code of {name} is '{country_code}'.")

The country code of norman wells is 'ca'.
The country code of guerrero negro is 'mx'.
The country code of dunedin is 'nz'.
The country code of lere is 'ng'.
The country code of chokwe is 'mz'.
The country code of tasiilaq is 'gl'.
The country code of energetik is 'ru'.
The country code of sinnamary is 'gf'.
The country code of punta arenas is 'cl'.
The country code of mataura is 'pf'.


In [4]:
# Create settings dictionary with information we're interested in
# settings = {"units": "metric", "appid": api_key}

In [5]:
# Get current weather
# current_weather_paris = owm.get_current("Paris", **settings)
# print(f"Current weather object for Paris: {current_weather_paris}.")

Current weather object for Paris: {'coord': {'lon': 2.35, 'lat': 48.86}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 1.24, 'pressure': 991, 'humidity': 94, 'temp_min': -2, 'temp_max': 3}, 'visibility': 10000, 'wind': {'speed': 1.5, 'deg': 50}, 'clouds': {'all': 0}, 'dt': 1520056800, 'sys': {'type': 1, 'id': 5610, 'message': 0.0044, 'country': 'FR', 'sunrise': 1520058503, 'sunset': 1520098642}, 'id': 2988507, 'name': 'Paris', 'cod': 200}.


In [6]:
# summary = ["name", "main.temp"]

# data = current_weather_paris(*summary)
# print(f"The current weather summary for Paris is: {data}.")

The current weather summary for Paris is: ('Paris', 1.24).
